In [140]:
import os

import pandas as pd
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from geopy.geocoders import GoogleV3, Nominatim
import folium
import geojson
import numpy as np
from shapely.geometry import shape, Point

In [8]:
load_dotenv()
GOOGLE_GEO_API_KEY = os.environ.get('GOOGLE_GEO_API_KEY')

In [9]:
def find_flat_in_minsk() -> list:
    _flats = []

    for page_number in range(1, 26):
        html_page_name = 'flat/%s.html' % page_number
        with open(html_page_name) as html_file:
            soup = BeautifulSoup(html_file, 'html.parser')

        prices = soup.find_all('span', attrs={'data-bind': "text: SearchApartments.formatPrice(apartment.price, 'USD')"})

        flats = soup.find_all('span', attrs={'class': 'classified__caption-item classified__caption-item_type', 
                                             'data-bind': 'text: SearchApartments.formatRentType(apartment.rent_type)'})

        addresses = soup.find_all('span', attrs={'class': 'classified__caption-item classified__caption-item_adress', 
                                                 'data-bind': 'text: apartment.location.user_address'})

        # print(page_number, '| prices:', len(prices), ', flats:', len(flats), ', addresses', len(addresses), '\n')

        for price, flat_size, address in zip(prices, flats, addresses):
            pr = int(price.text)
            fs = flat_size.text
            if fs == 'Комната':
                fs = 0.5
            else:
                fs = int(fs[0])

            ads = address.text

            _flats.append([pr, fs, ads])
    
    return _flats

In [10]:
# minsk_flats = find_flat_in_minsk()
print('Now avalible', len(minsk_flats), 'flats in Minsk')
minsk_flats[:5]

NameError: name 'minsk_flats' is not defined

In [11]:
def find_coords(flat_list: list):
    for flat in flat_list:
        try:
            address = flat[2] + ', Minsk'
            google_geo = GoogleV3(api_key=GOOGLE_GEO_API_KEY)
            coords = google_geo.geocode(address)

            lat = round(float(coords.latitude), 7)
            lng = round(float(coords.longitude), 7)

            flat.append(lat)
            flat.append(lng)
            # print(lat, lng)
        except Exception as ex:
            print('Exception in google_location: %s' % ex)

In [12]:
# find_coords(minsk_flats)

In [13]:
minsk_flats[:5]

NameError: name 'minsk_flats' is not defined

In [14]:
columns = ['price', 'flat-size', 'address', 'lat', 'lng']
df_flats = pd.DataFrame(data=minsk_flats, columns=columns)
df_flats.head()

NameError: name 'minsk_flats' is not defined

In [15]:
# df_flats.to_csv('minsk_flats.csv')

In [16]:
df_flats = pd.read_csv('minsk_flats.csv')
df_flats.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
print(df_flats.shape)
df_flats.head()

(900, 5)


,price,flat-size,address,lat,lng
0,295,1.0,"Чернышевского, 7",53.926984,27.600406
1,900,4.0,"Калинина, 7А",53.926420,27.605746
2,470,2.0,"Мельникайте, 16",53.909619,27.544978
3,380,1.0,"Академика Фёдорова, 3",53.872958,27.633268
4,300,2.0,"Волоха, 7 к1",53.896907,27.521841


In [71]:
address = 'Minsk, BY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Minsk are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Minsk are 53.902334, 27.5618791.


In [125]:
map_minsk_flat = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, price, flat_size, address in zip(df_flats['lat'], df_flats['lng'], df_flats['price'], df_flats['flat-size'], df_flats['address']):
    label = '{}$, {}k, {}'.format(price, flat_size, address)
    label_f = folium.Popup(label, parse_html=True)
    tooltip = folium.Tooltip(label)
    folium.CircleMarker(
        (lat, lng),
        radius=5,
        popup=label_f,
        tooltip=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_minsk_flat)  
    
map_minsk_flat

In [137]:
minsk_areas = r'minsk_areas.geojson'

In [150]:
def open_geojson():
    with open(minsk_areas, encoding="utf8") as f:
        data = geojson.load(f)
    
    point = Point(27.429495, 53.912448)
    
    count = 0
    public_service = []
    for feature_t in data['features']:
#         print(feature_t['properties'])
        neighborhood = feature_t['properties']['neighborhood']
        public_service.append(feature_t['properties']['neighborhood'])
        count += 1
        
        polygon = shape(feature_t['geometry'])
        if polygon.contains(point):
            print('Found containing polygon:', neighborhood)
            
    print(count)
    
    return public_service

In [151]:
minsk_areas_list = open_geojson()

Found containing polygon: Каменная горка
65


In [143]:
df_jes = pd.DataFrame(data=minsk_areas_list, columns=['area'])
df_jes.head()

,area
0,Каменная горка
1,Домбровка
2,кунцевщина
3,Красный бор
4,Запад


In [130]:
tdf = pd.DataFrame(data=np.random.randint(10, size=(130, 2)), columns=['price', 'rating'])
tdf = pd.concat([tdf, df_jes], axis=1)
tdf.head()

,price,rating,area
0,8,7,Каменная горка
1,9,0,Домбровка
2,9,7,кунцевщина
3,4,6,Красный бор
4,0,6,Запад


In [132]:
# map_minsk_flat = folium.Map(location=[latitude, longitude], zoom_start=12)

choropleth = folium.Choropleth(
    geo_data=minsk_areas,
    data=tdf,
    columns=['area', 'rating'],
    key_on='feature.properties.neighborhood',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Flat Price',
).add_to(map_minsk_flat)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['neighborhood'])
)

# display map
map_minsk_flat